In [1]:
import sys

In [2]:
print(sys.executable)

/home/hbcho991/.conda/envs/R-Adapter/bin/python


In [ ]:
sys.path.append("PATH")

In [3]:
!python --version

Python 3.11.5


In [ ]:
!ln -s /home/dataset/imagenet1k/data ./datasets/data/ILSVRC2012

In [5]:
import sys

In [6]:
print(sys.path)

['/home/hbcho991/.conda/envs/R-Adapter/lib/python310.zip', '/home/hbcho991/.conda/envs/R-Adapter/lib/python3.10', '/home/hbcho991/.conda/envs/R-Adapter/lib/python3.10/lib-dynload', '', '/home/hbcho991/.conda/envs/R-Adapter/lib/python3.10/site-packages', '/home/hbcho991/.conda/envs/R-Adapter/lib/python3.10/site-packages/setuptools/_vendor']


In [ ]:
sys.path.insert(0,'/home/hbcho991/R-Adapter/R-Adapter-ECCV2024/')

In [6]:
%load_ext autoreload
%autoreload 2

In [ ]:
!python datacreation_scripts/imagenet_csv_creator.py

In [ ]:
!pip install wandb

In [ ]:
!pip install git+https://github.com/modestyachts/ImageNetV2_pytorch

In [ ]:
!pip install open_clip_torch

In [ ]:
!pip install wilds braceexpand webdataset h5py

In [ ]:
!pip install pycocotools

## full-tuning 실행

In [4]:
!torchrun --master_port 29400 --nproc_per_node 4 -m src.main     --train-dataset=ImageNet     --epochs=10     --lr=5e-4     --wd=0     --workers=4     --batch-size=32     --model=ViT-B/32     --eval-datasets=ImageNet,ImageNetV2     --template=openai_imagenet_template     --save=./checkpoints/     --data-location=./datasets/data/     --ft_data=./datasets/csv/imagenet_classID.csv     --adapter=16,16     --drop-path=0.2     --ema=0.999     --eval-scale=0.5     --grad-clip-norm=1     --ls=0.02     --mg=0.05     --eval_epoch=0     --csv-img-key=filepath     --csv-caption-key=title     --supervised-label-key=class     --exp_name=ImageNet/R-Adapter_full_finetune

W1029 20:13:21.077000 22986400442176 torch/distributed/run.py:779] 
W1029 20:13:21.077000 22986400442176 torch/distributed/run.py:779] *****************************************
W1029 20:13:21.077000 22986400442176 torch/distributed/run.py:779] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W1029 20:13:21.077000 22986400442176 torch/distributed/run.py:779] *****************************************
/opt/anaconda3/2023.09-0/lib/python3.11/site-packages/transformers/utils/generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/opt/anaconda3/2023.09-0/lib/python3.11/site-packages/transformers/utils/generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Ple

/home/hbcho991/R-Adapter/R-Adapter-ECCV2024/src/main.py:288: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/home/hbcho991/R-Adapter/R-Adapter-ECCV2024/src/main.py:288: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/home/hbcho991/R-Adapter/R-Adapter-ECCV2024/src/main.py:288: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/home/hbcho991/R-Adapter/R-Adapter-ECCV2024/src/main.py:288: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
[rank0]:I1029 20:14:47.637000 22580217919296 torch/nn/parallel/distributed.py:1524] Reducer buckets have been rebuilt in this iteration.
[rank1]:I1029 20:14:47.642000 22669243107136 torch/

In [1]:
!nvidia-smi

Tue Oct 29 19:57:18 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        Off | 00000000:17:00.0 Off |                  Off |
| 33%   25C    P8              17W / 450W |      5MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## GMoE 실행

In [8]:
!python3 -m pip uninstall tutel -y

In [9]:
!python3 -m pip install --user --upgrade git+https://github.com/microsoft/tutel@main

  Cloning https://github.com/microsoft/tutel (to revision main) to /tmp/pip-req-build-r6u236_y
  Running command git clone --filter=blob:none --quiet https://github.com/microsoft/tutel /tmp/pip-req-build-r6u236_y
  Resolved https://github.com/microsoft/tutel to commit fc34436c54d98a5ed2422621a7f0a8e0a02458e5
  Preparing metadata (setup.py) ... done
  Created wheel for tutel: filename=tutel-0.3-cp311-cp311-linux_x86_64.whl size=165147 sha256=a6ba8774efd29407fa460d894bafe389bc50b74e133cddb20a68e5e8ad3eee19
  Stored in directory: /tmp/pip-ephem-wheel-cache-qcy736k7/wheels/39/41/87/a4f8853e23b074eafab1071b7fb2ba0ff2df1aa024294986f4
Successfully built tutel


In [10]:
!torchrun --master_port 29400 --nproc_per_node 4 -m src.main     --train-dataset=ImageNet     --epochs=10     --lr=5e-4     --wd=0     --workers=4     --batch-size=32     --model=ViT-B/32     --eval-datasets=ImageNet,ImageNetV2     --template=openai_imagenet_template     --save=./checkpoints/     --data-location=./datasets/data/     --ft_data=./datasets/csv/imagenet_classID.csv     --adapter=16,16     --drop-path=0.2     --ema=0.999     --eval-scale=0.5     --grad-clip-norm=1     --ls=0.02     --mg=0.05     --eval_epoch=0     --csv-img-key=filepath     --csv-caption-key=title     --supervised-label-key=class     --exp_name=ImageNet/R-Adapter_full_finetune

W1029 21:31:52.700000 22626981287744 torch/distributed/run.py:779] 
W1029 21:31:52.700000 22626981287744 torch/distributed/run.py:779] *****************************************
W1029 21:31:52.700000 22626981287744 torch/distributed/run.py:779] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W1029 21:31:52.700000 22626981287744 torch/distributed/run.py:779] *****************************************
/opt/anaconda3/2023.09-0/lib/python3.11/site-packages/transformers/utils/generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/opt/anaconda3/2023.09-0/lib/python3.11/site-packages/transformers/utils/generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Ple

E1029 21:31:56.788000 22626981287744 torch/distributed/elastic/multiprocessing/api.py:833] failed (exitcode: 1) local_rank: 3 (pid: 2280793) of binary: /opt/anaconda3/2023.09-0/bin/python
Traceback (most recent call last):
  File "/home/hbcho991/.local/bin/torchrun", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/home/hbcho991/.local/lib/python3.11/site-packages/torch/distributed/elastic/multiprocessing/errors/__init__.py", line 348, in wrapper
    return f(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^
  File "/home/hbcho991/.local/lib/python3.11/site-packages/torch/distributed/run.py", line 901, in main
    run(args)
  File "/home/hbcho991/.local/lib/python3.11/site-packages/torch/distributed/run.py", line 892, in run
    elastic_launch(
  File "/home/hbcho991/.local/lib/python3.11/site-packages/torch/distributed/launcher/api.py", line 133, in __call__
    return launch_agent(self._config, self._entrypoint, list(args))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [6]:
from tutel import moe as tutel_moe

ImportError: cannot import name 'tutel_custom_kernel' from 'tutel.impls.jit_compiler' (/home/hbcho991/.local/lib/python3.10/site-packages/tutel/impls/jit_compiler.py)

## few-shot 실행

In [ ]:
!torchrun --master_port 29400 --nproc_per_node 4 -m src.few_shot     --train-dataset=ImageNet     --epochs=50     --lr=5e-4     --wd=0     --workers=4     --batch-size=32     --model=ViT-B/16     --eval-datasets=ImageNet,ImageNetV2,ImageNetR,ImageNetSketch,ImageNetA     --template=openai_imagenet_template     --save=./checkpoints/     --data-location=./datasets/data/     --ft_data=./datasets/csv/imagenet_classID.csv     --adapter=4,4     --drop-path=0.2     --ema=0.999     --eval-scale=0.5     --grad-clip-norm=1     --ls=0.02     --mg=0.05     --csv-img-key=filepath     --csv-caption-key=title     --supervised-label-key=class     --warmup_length=0     --k=16     --exp_name=ImageNet/R-Adapter_few_shot_16shot


## base-to-novel 실행

In [ ]:
!ln -s /home/dataset/flowers ./datasets/data/flowers102

In [ ]:
!torchrun --master_port 29400 --nproc_per_node 4 -m src.main_few_shot_novel         --train-dataset=flowers102        --epochs=101         --lr=5e-4         --wd=0         --workers=4         --batch-size=32         --model=ViT-B/16         --template=openai_imagenet_template         --save=./checkpoints/         --data-location=./datasets/data/         --adapter=4,4         --drop-path=0.2         --ema=0.9         --eval-scale=0.5         --grad-clip-norm=1         --ls=0         --mg=0.1         --csv-img-key=filepath         --csv-caption-key=title         --supervised-label-key=class         --warmup_length=500         --k=16         --save=False         --exp_name=${dataset}/R-Adapter_few_shot_16shot